<a href="https://colab.research.google.com/github/Tung0000/Test_model/blob/main/Tsest_Model_Drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [1]:
!pip install ultralytics
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
from google.colab import files
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
uploaded = files.upload()  # เรียกใช้ตัวอัพโหลดรูปของ Colab
if not uploaded:
    raise FileNotFoundError("No file uploaded!")
test_image_path = next(iter(uploaded.keys()))  # เก็บชื่อไฟล์เข้าสู่ตัวแปร

Saving drones-over-city-stockcake.jpg to drones-over-city-stockcake.jpg


In [6]:
with open("bestmore.pt", "rb") as f:
    print(f.read(4))


b'PK\x03\x04'


In [7]:
# โหลดโมเดล best
model_path = 'bestmore.pt'
if not os.path.exists(model_path):
    raise FileNotFoundError(f"YOLO model file not found at {model_path}")
model = YOLO(model_path)

In [8]:
# โหลดภาพด้วย OpenCV
image = cv2.imread(test_image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at {test_image_path}")

# แปลงรูปสีเป็นขาวดำ
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# ส่งภาพเข้าโมเดล
results = model(image)


0: 448x800 13 drones, 827.9ms
Speed: 20.7ms preprocess, 827.9ms inference, 40.5ms postprocess per image at shape (1, 3, 448, 800)


In [9]:
for result in results:
    boxes = result.boxes.cpu().numpy()  # แปลงตำแหน่ง Bounding Box ให้อยู่ในรูปแบบ NumPy array
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # ดึงค่าพิกัด Bounding Box (มุมซ้ายบนและมุมขวาล่าง)
        confidence = box.conf[0]  # ดึงค่าความเชื่อมั่นวัตถุ
        cls = int(box.cls[0])  # ดึงค่า class id ของวัตถุที่ตรวจจับได้
        if confidence > 0.5:  # กรองเฉพาะวัตถุที่มีเชื่อมั่นมากกว่า 50%
            label = f"{model.names[cls]} {confidence:.2f}"  # สร้างข้อความแสดงชนิดของวัตถุและค่าความเชื่อมั่น
            # ตีกรอบ (สีแดงและความหนา 3 พิกเซล)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3)
            # เขียนข้อความ label
            cv2.putText(image,label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

In [38]:
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>

In [16]:
from ultralytics import YOLO
import csv
from pathlib import Path

# โหลดโมเดล
model = YOLO("bestmore.pt")

# path ที่จะเซฟผลลัพธ์ CSV
csv_path = "results.csv"

# ถ้ายังไม่มีไฟล์ csv ให้สร้าง header
if not Path(csv_path).exists():
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "class", "confidence", "x_min", "y_min", "x_max", "y_max"])

# ทำนายทั้งรูปหรือวิดีโอ
results = model("fin.mp4", save=True)   # <-- ใส่ได้ทั้ง jpg, png, mp4, avi, webcam(0)

# loop เก็บผลลัพธ์
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()   # [x_min, y_min, x_max, y_max]
    confs = r.boxes.conf.cpu().numpy()   # confidence
    classes = r.boxes.cls.cpu().numpy()  # class id

    for box, conf, cls in zip(boxes, confs, classes):
        with open(csv_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                r.path if hasattr(r, "path") else "video_frame",
                model.names[int(cls)],
                float(conf),
                int(box[0]), int(box[1]), int(box[2]), int(box[3])
            ])



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2477) /content/fin.mp4: 448x800 1 drone, 596.8ms
video 1/1 (frame 2/2477) /content/fin.mp4: 448x800 1 drone, 646.8ms
video 1/1 (frame 3/2477) /content/fin.mp4: 448x800 1 drone, 536.9ms
video 1/1 (frame 4/2477) /content/fin.mp4: 448x800 1 drone, 551.3ms
video 1/1 (frame 5/2477) /content/fin.mp4: 448x800 1 drone, 529.3ms
video 1/1 (frame 6/2477) /content/fin.mp4: 448x800 1 drone, 563.5ms
video 1/1 (frame 7/2477) /content/fin.mp4: 448x8